<center><img src = "https://klgadgetguy.com/wp-content/uploads/2018/10/6ce1f4f6d79353c5f24ee047a5132d77.jpg" width = "750" height = "500"/></center>  

# The Goal of this Competition
Finding near-duplicates in large datasets is an important problem for many online businesses. Our task is to identify which products have been posted repeatedly. 

# Evaluation Criteria
Submissions will be evaluated based on their mean F1 score. The mean is calculated in a sample-wise fashion, meaning that an [F1 score](https://deepai.org/machine-learning-glossary-and-terms/f-score) is calculated for every predicted row, then averaged.

<h2 style='color:white; background:#000080; border:0'><center>Overview</center></h2>

What is Shopee ?
[Wikipedia page](https://en.wikipedia.org/wiki/Shopee) says that Shopee Pte Ltd (/ʃɒpiː/ SHO-pee) is a Singaporean multinational technology company which focuses mainly on e-commerce. Headquartered under Sea Group (previously known as Garena),[2] Shopee was first launched in Singapore in 2015, and later expanded its reach to Malaysia, Thailand, Taiwan, Indonesia, Vietnam, the Philippines and Brazil.

[The competition page](https://www.kaggle.com/c/shopee-product-matching/data) states that Shopee is the leading e-commerce platform in Southeast Asia and Taiwan. Customers appreciate its easy, secure, and fast online shopping experience tailored to their region. The company also provides strong payment and logistical support along with a 'Lowest Price Guaranteed' feature on thousands of Shopee's listed products.

[The competition page](https://www.kaggle.com/c/shopee-product-matching/data) states the following about each ID.
Files
[train/test].csv - the training set metadata. Each row contains the data for a single posting. Multiple postings might have the exact same image ID, but with different titles or vice versa.

* posting_id - the ID code for the posting.

* image - the image id/md5sum.

* image_phash - a perceptual hash of the image.

* title - the product description for the posting.

* label_group - ID code for all postings that map to the same product. Not provided for the test set.

* [train/test]images - the images associated with the postings.

* sample_submission.csv - a sample submission file in the correct format.

* posting_id - the ID code for the posting.

* matches - Space delimited list of all posting IDs that match this posting. Posts always self-match. Group sizes were capped at 50, so there's no need to predict more than 50 matches.

In [ ]:
pip install textfeatures

In [ ]:
pip install nltk

In [ ]:
import nltk

* NLTK (Natural Language Toolkit) is useful for natural language processing. The official documentation is [here](http://www.nltk.org/).

In [ ]:
import plotly.express as px

* Prepare a library to draw a histogram.

In [ ]:
!pip install hvplot
import hvplot.pandas  # custom install

from glob import glob

from bq_helper import BigQueryHelper
from dask import bag, diagnostics 
from urllib import request

import missingno as msno

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import pydicom
import matplotlib.pyplot as plt

# import useful tools
from glob import glob
from PIL import Image
import cv2
import pydicom as dcm
import random
import matplotlib.patches as patches
from sklearn.model_selection import KFold
from pydicom.pixel_data_handlers.util import apply_voi_lut

from sklearn.model_selection import StratifiedKFold
import warnings

import plotly.express as px
from wordcloud import WordCloud, STOPWORDS

# Work with phash
import imagehash

# import data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import matplotlib
import pydicom as dicom

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.models.widgets import Tabs
import skimage.io as io

# import data augmentation
import albumentations as albu

# import math module
import math

# Libraries
import pandas_profiling
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
import matplotlib.patches as patches
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import tqdm
from tqdm.auto import tqdm as tqdmp
tqdmp.pandas()

# One-hot encoding
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Other
from random import randint
import warnings
import csv
warnings.filterwarnings("ignore")

In [ ]:
# Color scheme
colors = ["#EDAC54", "#F4C5B7", "#DD7555", "#B95F18", "#000080"]

<h2 style='color:white; background:#000080; border:0'><center>Usefull functuions</center></h2>

In [ ]:
def plot_images(images_number):
    
    plot_list = train_df['image'].sample(n=images_number).tolist()
    size = np.sqrt(images_number)
    if int(size)*int(size) < images_number:
        size = int(size) + 1
        
    plt.figure(figsize=(10, 10))
    
    ind=0
    for image_id in plot_list:
        plt.subplot(size, size, ind + 1)
        image = cv2.imread(os.path.join('../input/shopee-product-matching/train_images/', image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(image_id, fontsize=12)
        plt.axis("off")
        ind+=1
    plt.show()

<h2 style='color:white; background:#000080; border:0'><center>Loading data</center></h2>

In [ ]:
# Setup the paths to train and test images

TEST_DIR = "../input/shopee-product-matching/test_images/"
TRAIN_DIR = "../input/shopee-product-matching/train_images/"
dataset_dir = "../input/shopee-product-matching/"

In [ ]:
# Glob the directories and get the lists of train and test images
train_fns = glob(TRAIN_DIR + '*')
test_fns = glob(TEST_DIR + '*')

# Loading training data and test data
train_df = pd.read_csv(dataset_dir+'train.csv')
test = pd.read_csv(dataset_dir+'test.csv')
sample = pd.read_csv(dataset_dir+'sample_submission.csv')

In [ ]:
train_images = dataset_dir + "/train_images/" + train_df['image']
train_df['path'] = train_images

test_images = dataset_dir + "/test_images/" + test['image']
test['path'] = test_images

<h2 style='color:white; background:#000080; border:0'><center>Statistics</center></h2>

In [ ]:
# Let's find out how many images are under the directory
print('Train images: %d' %len(os.listdir(os.path.join(dataset_dir, "train_images"))))
print('Test images: %d' %len(os.listdir(os.path.join(dataset_dir, "test_images"))))

Let's figure out the unique number for each columns.

In [ ]:
train_df.head(5)

In [ ]:
train_df.info()

In [ ]:
train_df.nunique().to_frame().rename(columns={0:"Unique Values"}).style.background_gradient(cmap="plasma")

* The number of uniqueness is the same for image and path. image_phash and label_group are fewer in number than image, so there is a possibility to aggregate common images with these.

In [ ]:
# Display of training data
print(train_df)

In [ ]:
# Check for missing values in the training data
train_df.isnull().sum()

* Fortunately, it turns out there are no missing values.

* Understanding same label_groups
* Counting the number of images in each label group

In [ ]:
labels = train_df.groupby("label_group")["image"].count().reset_index()
labels.columns=["label_group","image_num"]
labels

In [ ]:
sortlabels = labels.sort_values("image_num")
sortlabels

* minimum images are 2, max images are 51

# Display Duplicated Items from Train Data
Using the column label_group which is the ground truth, we can display examples of duplicated items.

In [ ]:
BASE = '../input/shopee-product-matching/train_images/'

def displayDF(train_df, random=False, COLS=6, ROWS=4, path=BASE):
    for k in range(ROWS):
        plt.figure(figsize=(20,5))
        for j in range(COLS):
            if random: row = np.random.randint(0,len(train_df))
            else: row = COLS*k + j
            name = train_df.iloc[row,1]
            title = train_df.iloc[row,3]
            title_with_return = ""
            for i,ch in enumerate(title):
                title_with_return += ch
                if (i!=0)&(i%20==0): title_with_return += '\n'
            img = cv2.imread(path+name)
            plt.subplot(1,COLS,j+1)
            plt.title(title_with_return)
            plt.axis('off')
            plt.imshow(img)
        plt.show()
        
displayDF(train_df,random=True)

In [ ]:
groups = train_df.label_group.value_counts()
plt.figure(figsize=(20,5))
plt.plot(np.arange(len(groups)),groups.values)
plt.ylabel('Duplicate Count',size=14)
plt.xlabel('Index of Unique Item',size=14)
plt.title('Duplicate Count vs. Unique Item Count',size=16)
plt.show()

plt.figure(figsize=(20,5))
plt.bar(groups.index.values[:50].astype('str'),groups.values[:50])
plt.xticks(rotation = 45)
plt.ylabel('Duplicate Count',size=14)
plt.xlabel('Label Group',size=14)
plt.title('Top 50 Duplicated Items',size=16)
plt.show()

In [ ]:
for k in range(5):
    print('#'*40)
    print('### TOP %i DUPLICATED ITEM:'%(k+1),groups.index[k])
    print('#'*40)
    top = train_df.loc[train_df.label_group==groups.index[k]]
    displayDF(top, random=False, ROWS=2, COLS=4)

# test

In [ ]:
# Display of test data
print(test)

# sample

In [ ]:
# Display of sample data
print(sample)

[The competition page](https://www.kaggle.com/c/shopee-product-matching/data) states the following about the IDs of the submitted file.

* sample_submission.csv - a sample submission file in the correct format.

* posting_id - the ID code for the posting.

matches - Space delimited list of all posting IDs that match this posting. Posts always self-match. Group sizes were capped at 50, so there's no need to predict more than 50 matches.**

In [ ]:
plt.figure(figsize=(8,8))

for num,a in enumerate(test["path"]):
    
    plt.subplot(1,3,num+1)
    img = cv2.imread(a)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img)
    plt.axis("off")

In [ ]:
plot_images(8)

<h2 style='color:white; background:#000080; border:0'><center>Data Visualization</center></h2>

Image Label Groups by No. of Images

In [ ]:
top10_names = train_df['label_group'].value_counts().index.tolist()[:15]
top10_values = train_df['label_group'].value_counts().tolist()[:15]

plt.figure(figsize=(20, 10))
sns.barplot(x=top10_names, y=top10_values)
plt.xticks(rotation=45)
plt.xlabel("Label Group")
plt.ylabel("Image Count")
plt.title("Top-15 Label Groups by Image Count")
plt.show()

* Take a look at the distribution of the number of images in Label Groups. There are some groups with a small number of images, but there are neither groups with a large number of images nor groups with a small number of images.

[](http://)<h2 style='color:white; background:#000080; border:0'><center>Titles analysis</center></h2>

In [ ]:
stopwords = set(STOPWORDS) 
wordcloud = WordCloud(width = 800, 
                      height = 800,
                      background_color ='white',
                      min_font_size = 10,
                      stopwords = stopwords,).generate(' '.join(train_df['title'])) 

# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 

Basic NLP

In [ ]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True):

    lst_stopwords = nltk.corpus.stopwords.words("english")
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()    
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
#Clean Address
train_df["clean_title"] = train_df["title"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, ))

In [ ]:
#Length of Title
train_df['clean_title_len'] = train_df['clean_title'].apply(lambda x: len(x))

In [ ]:
#Word Count
train_df['clean_title_word_count'] =train_df["clean_title"].apply(lambda x: len(str(x).split(" ")))

In [ ]:
#Character Count
train_df['clean_title_char_count'] = train_df["clean_title"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

In [ ]:
#Average Word Length
train_df['clean_title_avg_word_length'] = train_df['clean_title_char_count'] / train_df['clean_title_word_count']

In [ ]:
def plot_distribution(x, title):

    fig = px.histogram(
    train_df, 
    x = x,
    width = 800,
    height = 500,
    title = title
    )
    
    fig.show()

In [ ]:
plot_distribution(x = 'clean_title_len', title = 'Title Length Distribution')

In [ ]:
plot_distribution(x = 'clean_title_word_count', title = 'Word Count Distribution')

In [ ]:
plot_distribution(x = 'clean_title_char_count', title = 'Character Count Distribution')

In [ ]:
plot_distribution(x = 'clean_title_avg_word_length', title = 'Average Word Length Distribution')

<h2 style='color:white; background:#000080; border:0'><center>Images analysis</center></h2>

In [ ]:
phashgroup = train_df.groupby("image_phash")["path"].count().reset_index()
phashgroup.columns=["image_phash","counts"]
phashgroup

In [ ]:
sortphash = phashgroup.sort_values("counts")
sortphash

* maximum counts are 26.

In [ ]:
tmpdf = train_df[train_df["image_phash"]==sortphash["image_phash"].iloc[-1]]
tmpdf

In [ ]:
# Shape columns
train_df['img_shape'] = train_df['path'].progress_apply(lambda x: np.shape(io.imread(x)))

* Draw the height, width, and color of the image as a scatter plot with histogram.

In [ ]:
shapes = pd.DataFrame().from_records(train_df['img_shape'])
shapes.columns = ['Width', 'Height', 'Colors']

sns.set_style("white")
sns.jointplot(x = shapes.iloc[:, 0].astype('float32'), 
              y = shapes.iloc[:, 1].astype('float32'),
              height = 8, color = '#000080')
plt.show()

Duplicated Images

In [ ]:
# Get the count of apparitions per image
image_count = train_df["image"].value_counts().reset_index()
image_count.columns = ["image", "count"]
image_count_duplicates = image_count[image_count["count"] > 1]
print("Total no. of images with duplicates: {:,}".format(len(image_count_duplicates)))

#Plot
fig, ax = plt.subplots(figsize=(16, 7))
plt.bar(x=image_count_duplicates.iloc[::16]["image"],
        height=image_count_duplicates.iloc[::16]["count"],
        color=colors[4])
plt.title("Duplicated Images: How many apparitions?", fontsize=20)
plt.xticks([])
plt.xlabel("Image ID", fontsize=16)
plt.ylabel("Count", fontsize=16);

* There are 1,246 images that have 2 or more apparitions
* the title differs for most of them
* the label_group differs for most of them as well

In [ ]:
def image_viz(image_path):
    """
    Function for visualization.
    Takes path to image as input.
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    plt.imshow(img)
    plt.axis('off')

In [ ]:
def match_matrix(phash_array):
    """
    A function that checks for matches by phash value.
    Takes phash values as input.
    Output - phash diff matrix (pandas data frame)
    """
    phashs = phash_array.apply(lambda x: imagehash.hex_to_hash(x))
    phash_matrix = pd.DataFrame()
    pbar = tqdm.tqdm(total = len(phash_array), desc = 'Progress:', 
                     position = 0, leave = True)
    for idx, i in enumerate(phash_array):
        pbar.update(1)
        phash_matrix = pd.concat([phash_matrix, phashs - imagehash.hex_to_hash(i)], 
                                 axis = 1)
    pbar.close()
    phash_matrix.columns = range(len(phash_array))
    return phash_matrix

* Check the fit by Phash value.

In [ ]:
train_part = train_df.iloc[:1000, :]
matches = match_matrix(train_part['image_phash'])
matches

In [ ]:
test_match = match_matrix(test['image_phash'][:3])
test_match

In [ ]:
train_part.loc[[11,12],['title']]

* Posting_id is different, but title, image_phash and label_group are the same image.

In [ ]:
plt.figure(figsize = (15, 10))
for idx, i in enumerate([train_part.loc[11, 'path'], 
                         train_part.loc[12, 'path']]):
    plt.subplot(1, 2, idx + 1)
    image_viz(i)
plt.show()

In [ ]:
train_part.loc[[889,890,891],['posting_id','image_phash','title','label_group']]

* Posting_id and title are different, but image_phash and label_group are the same image.

In [ ]:
plt.figure(figsize = (15, 10))
for idx, i in enumerate([train_part.loc[889, 'path'], 
                         train_part.loc[890, 'path'], 
                         train_part.loc[891, 'path']]):
    plt.subplot(1, 3, idx + 1)
    image_viz(i)
plt.show()

In [ ]:
plt.figure(figsize = (15, 10))
for idx, i in enumerate([train_part.loc[997, 'path'], 
                         train_part.loc[520, 'path']]):
    plt.subplot(1, 2, idx + 1)
    image_viz(i)
plt.show()

* An example of a similar, but not identical, image is shown below.

In [ ]:
plt.figure(figsize = (15, 10))
for idx, i in enumerate([train_part.loc[55, 'path'], 
                         train_part.loc[312, 'path']]):
    plt.subplot(1, 2, idx + 1)
    image_viz(i)
plt.show()

* An example of a similar, but not identical, image is shown below.

In [ ]:
plt.figure(figsize = (15, 10))
for idx, i in enumerate([train_part.loc[128, 'path'], 
                         train_part.loc[515, 'path']]):
    plt.subplot(1, 2, idx + 1)
    image_viz(i)
plt.show()

* An example of a similar, but not identical, image is shown below.

In [ ]:
plt.figure(figsize = (15, 10))
for idx, i in enumerate([train_part.loc[216, 'path'], 
                         train_part.loc[567, 'path']]):
    plt.subplot(1, 2, idx + 1)
    image_viz(i)
plt.show()

<h2 style='color:white; background:#000080; border:0'><center>Test Image</center></h2>

In [ ]:
plt.figure(figsize = (15, 10))
for idx, i in enumerate(test['path']):
    plt.subplot(1, 3, idx + 1)
    image_viz(i)
plt.show()

In [ ]:
test

In [ ]:
# coding: utf-8
from tqdm import tqdm
import time

# Set the total value 
bar = tqdm(total = 1000)
# Add description
bar.set_description('Progress rate')
for i in range(100):
    # Set the progress
    bar.update(25)
    time.sleep(1)

<h2 style='color:white; background:#000080; border:0'><center>Acknowledgements</center></h2>

* [Shopee - Data understanding and analysis](https://www.kaggle.com/isaienkov/shopee-data-understanding-and-analysis)
* [Shopee: Before we start (EDA, PHASH, Baseline)](https://www.kaggle.com/maksymshkliarevskyi/shopee-before-we-start-eda-phash-baseline)
* [EDA of Shopee for starter](https://www.kaggle.com/chumajin/eda-of-shopee-for-starter)
* [Shopee: text prep | FE | image augmentation](https://www.kaggle.com/andradaolteanu/shopee-text-prep-fe-image-augmentation)
* [[V5]Shopee InDepth EDA:One stop for all your needs](https://www.kaggle.com/ishandutta/v5-shopee-indepth-eda-one-stop-for-all-your-needs)
* [[V7]Shopee InDepth EDA:One stop for all your needs](https://www.kaggle.com/ishandutta/v7-shopee-indepth-eda-one-stop-for-all-your-needs)
* [Shopee: Before we start (EDA, PHASH, Baseline)](https://www.kaggle.com/maksymshkliarevskyi/shopee-before-we-start-eda-phash-baseline)
* [RAPIDS cuML TfidfVectorizer and KNN](https://www.kaggle.com/cdeotte/rapids-cuml-tfidfvectorizer-and-knn)

# Work in progress…

# Your upvote is my motivation